# Download Model and upload to Object Bucket

### Get Packages

In [74]:
!pip install --upgrade huggingface_hub -q
!pip install transformers -q
!pip install s3cmd -q

import torch
import os
import boto3


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


### Login to Hugging Face

In [41]:
!huggingface-cli login --token $huggingfacetoken

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /opt/app-root/src/.cache/huggingface/token
Login successful


### Check GPU

In [65]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

True
1
NVIDIA A10G


### Download Model from HuggingFace

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

/opt/app-root/lib64/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [73]:
# Default location of download
!ls ~/.cache/huggingface/hub/

models--mistralai--Mistral-7B-Instruct-v0.2  version.txt


### Save model to a folder

In [44]:
!pwd
save_dir = "/opt/app-root/src/mistral_model"

/opt/app-root/src/openshift-notebooks/notebooks


In [ ]:
# Save model and tokenizer locally
save_dir = "/opt/app-root/src/mistral_model"
os.makedirs(save_dir, exist_ok=True)
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

### To check S3 values

In [62]:
# !echo $AWS_ACCESS_KEY_ID
# !echo $AWS_SECRET_ACCESS_KEY
# !echo $AWS_S3_BUCKET
# !echo $AWS_S3_ENDPOINT
# !echo $AWS_DEFAULT_REGION

## Set up minio CLI

In [1]:
!curl https://dl.min.io/client/mc/release/linux-amd64/mc --create-dirs -o ./minio-binaries/mc
!chmod +x ./minio-binaries/mc
!export PATH=$PATH:/opt/app-root/src/openshift-notebooks/notebooks/minio-binaries # Does not work from jupyter notebook

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 25.2M  100 25.2M    0     0  18.1M      0  0:00:01  0:00:01 --:--:-- 18.1M
]11;?\Added `minio` successfully.
]11;?\minio
  URL       : http://minio-service.minio.svc.cluster.local:9000
  AccessKey : minio
  SecretKey : minio123
  API       : S3v4
  Path      : auto



In [2]:
!./minio-binaries/mc alias set minio http://minio-service.minio.svc.cluster.local:9000 --api "S3v4" minio minio123
!./minio-binaries/mc alias ls minio

]11;?\Added `minio` successfully.
]11;?\minio
  URL       : http://minio-service.minio.svc.cluster.local:9000
  AccessKey : minio
  SecretKey : minio123
  API       : S3v4
  Path      : auto



### Upload model to the bucket

In [4]:
save_dir = "/opt/app-root/src/mistral_model"
!./minio-binaries/mc cp -r $save_dir minio/model/

...onfig.json: 26.98 GiB / 26.98 GiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 65.62 MiB/s 7m0s

### Verify

In [61]:
!./minio-binaries/mc ls minio/model/

]11;?\[2024-05-04 14:05:53 UTC]   652B STANDARD config.json
[2024-05-04 14:05:53 UTC]   111B STANDARD generation_config.json
[2024-05-04 14:15:57 UTC] 4.6GiB STANDARD model-00001-of-00006.safetensors
[2024-05-04 14:15:55 UTC] 4.6GiB STANDARD model-00002-of-00006.safetensors
[2024-05-04 14:15:59 UTC] 4.7GiB STANDARD model-00003-of-00006.safetensors
[2024-05-04 14:15:58 UTC] 4.7GiB STANDARD model-00004-of-00006.safetensors
[2024-05-04 14:15:50 UTC] 4.5GiB STANDARD model-00005-of-00006.safetensors
[2024-05-04 14:14:51 UTC] 4.0GiB STANDARD model-00006-of-00006.safetensors
[2024-05-04 14:05:53 UTC]  23KiB STANDARD model.safetensors.index.json
[2024-05-04 14:05:53 UTC]   414B STANDARD special_tokens_map.json
[2024-05-04 14:05:54 UTC] 1.7MiB STANDARD tokenizer.json
[2024-05-04 14:05:53 UTC] 1.4KiB STANDARD tokenizer_config.json


# ROUGH